In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import itertools

ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
if ROOT_DIR not in sys.path: sys.path.append(ROOT_DIR)

import numpy as np
import proplot as plot
import tensorflow as tf
import pandas as pd

import DeepSparseCoding.tf1x.analysis.analysis_picker as ap
import DeepSparseCoding.tf1x.data.data_selector as ds
import DeepSparseCoding.tf1x.params.param_picker as pp
import DeepSparseCoding.tf1x.models.model_picker as mp
import DeepSparseCoding.utils.plot_functions as pf

rand_seed = 123
rand_state = np.random.RandomState(rand_seed)

color_vals = dict(zip(["blk", "lt_green", "md_green", "dk_green", "lt_blue", "md_blue", "dk_blue", "lt_red", "md_red", "dk_red"],
  ["#000000", "#A9DFBF", "#196F3D", "#27AE60", "#AED6F1", "#3498DB", "#21618C", "#F5B7B1", "#E74C3C", "#943126"]))

### Load DeepSparseCoding analyzer

In [ ]:
class params(object):
  def __init__(self):
    #self.device = "/cpu"
    self.device = "/gpu:0"
    self.analysis_dataset = "test"
    self.save_info = "analysis_" + self.analysis_dataset
    self.overwrite_analysis_log = False
    self.do_class_adversaries = False
    self.do_run_analysis = False
    self.do_evals = False
    self.do_basis_analysis = False
    self.do_inference = False
    self.do_atas = False 
    self.do_recon_adversaries = False
    self.do_neuron_visualization = False
    self.do_full_recon = False
    self.do_orientation_analysis = False 
    self.do_group_recons = False
    
    self.data_dir = os.path.join(ROOT_DIR, 'Datasets')
    self.data_type = 'vanhateren'
    self.vectorize_data = True
    self.rescale_data = False
    self.standardize_data = False
    self.contrast_normalize = False
    self.whiten_data = True
    self.whiten_method = "FT"
    self.whiten_batch_size = 2
    self.extract_patches = True
    self.num_patches = 1e5
    self.patch_edge_size = 16
    self.overlapping_patches = True
    self.randomize_patches = True
    self.patch_variance_threshold = 0.0
    self.lpf_data = False # whitening automatically includes lpf
    self.lpf_cutoff = 0.7
    self.batch_size = 100
    self.rand_seed = rand_seed
    self.rand_state = rand_state

In [ ]:
analysis_params = params()
#analysis_params.projects_dir = os.path.expanduser("~")+"/Redwood/JOV_Paper/Projects_New/"
analysis_params.projects_dir = os.path.expanduser("~")+"/Work/Projects/"

model_names = ['lca_512_vh', 'lca_1024_vh', 'lca_2560_vh']#, 'sae_768_vh', 'rica_768_vh']
model_types = ['LCA', 'LCA', 'LCA']#, 'SAE', 'ICA']
model_labels = ['2x', '4x', '10x']#, 'Sparse Autoencoder', 'Linear Autoencoder']
analyzers = []
for model_type, model_name, model_label in zip(model_types, model_names, model_labels):
    analysis_params.model_name = model_name
    analysis_params.version = '0.0'
    analysis_params.model_dir = analysis_params.projects_dir+analysis_params.model_name
    model_log_file = (analysis_params.model_dir+"/logfiles/"+analysis_params.model_name
      +"_v"+analysis_params.version+".log")
    analysis_params.model_type = model_type
    analyzer = ap.get_analyzer(analysis_params.model_type)
    analysis_params.save_info = "analysis_selectivity"
    analyzer.setup(analysis_params)
    analyzer.model_label = model_label
    analyzer.model_type = model_type
    analyzer.setup_model(analyzer.model_params)
    analyzer.load_analysis(save_info="analysis_train_kurakin_targeted")
    analyzer.nat_selectivity = {}
    analyzers.append(analyzer)

### Load data, weights, and activations

In [ ]:
data = ds.get_data(analysis_params)
data = analyzers[0].model.preprocess_dataset(data, analysis_params)
data = analyzers[0].model.reshape_dataset(data, analysis_params)

In [ ]:
num_imgs = int(analysis_params.num_patches)
#num_imgs = int(analysis_params.batch_size)

In [ ]:
num_imgs_test = 6
img_idx = np.random.randint(num_imgs-num_imgs_test)
fig, axs = plot.subplots(ncols=num_imgs_test)
for inc_img in range(num_imgs_test):
    im = axs[inc_img].imshow(data['train'].images[img_idx+inc_img,...].reshape(16, 16), cmap='greys_r')
axs.format(suptitle=f'DSC van hateren example images')
pf.clear_axes(axs)
plot.show()

In [ ]:
weights = []
for analyzer in analyzers:
    if analyzer.model_type == 'LCA':
        print(f'Loading {analyzer.analysis_params.cp_loc} from {analyzer.model_label}')
        weights.append(np.squeeze(analyzer.eval_analysis(data['train'].images[0,...][None,...],
            ['lca/weights/w:0'], analyzer.analysis_params.save_info)['lca/weights/w:0']))
#weights = [np.squeeze(analyzer.eval_analysis(data['train'].images[0,...][None,...], ['lca/weights/w:0'], analyzer.analysis_params.save_info)['lca/weights/w:0']) for analyzer in analyzers if analyzer.model_type=='LCA']

In [ ]:
num_plots_per_model = 6
fig, axs = plot.subplots(ncols=num_plots_per_model, nrows=len(analyzers))
for analyzer_idx, analyzer in enumerate(analyzers):
    ax_row = analyzer_idx
    weight_indices = np.random.randint(0, analyzer.model_params.num_neurons, num_plots_per_model)
    for ax_col, weight_idx in enumerate(weight_indices):
        im = axs[ax_row, ax_col].imshow(weights[analyzer_idx][:, weight_idx].reshape(16, 16), cmap='greys_r')
        axs[ax_row, 0].format(title=f'{analyzer.model_label} overcomplete')
    axs.format(suptitle=f'Model weights')
    pf.clear_axes(axs)
plot.show()

In [ ]:
indices = np.random.randint(low=0, high=analyzers[0].model.params.num_neurons, size=3)
fig, axs = plot.subplots(ncols=3, nrows=2)
for fig_idx, neuron_idx in enumerate(indices):
    axs[0, fig_idx].imshow(analyzers[0].bf_stats['basis_functions'][neuron_idx], cmap='greys_r')
    axs[0, fig_idx].format(title=f'Diameter = {analyzers[0].bf_stats["diameters"][neuron_idx]:.2f}')
    axs[1, fig_idx].imshow(analyzers[0].bf_stats['envelopes'][neuron_idx], cmap='greys_r')
pf.clear_axes(axs)
plot.show()

In [ ]:
lca_activations = [np.squeeze(analyzer.compute_activations(data['train'].images[0:num_imgs,...],
    activation_operation=analyzer.model.get_encodings))
    for analyzer in analyzers]

In [ ]:
  def compute_lambda_activations(images, model, weights, batch_size=None, activation_operation=None):
    """
    Computes the output code for a set of images.
    Outputs:
      evaluated activation_operation on the input images
    Inputs:
      images [np.ndarray] of shape (num_imgs, num_img_pixels)
      batch_size [int] how many inputs to use in a batch
      activation_operation [tf operation] that produces the output activation
        if None then it defaults to `model.get_encodings()`
    """
    if activation_operation is None:
        activation_operation = model.get_encodings
    images_shape = list(images.shape)
    num_images = images_shape[0]
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.compat.v1.Session(config=config, graph=model.graph) as sess:
        if batch_size is not None and batch_size < num_images:
            assert num_images % batch_size == 0, (
                "batch_size=%g must divide evenly into num_images=%g"%(batch_size, num_images))
            num_batches = int(np.ceil(num_images / batch_size))
            batch_image_shape = [batch_size] + images_shape[1:]
            sess.run(model.init_op, {model.input_placeholder:np.zeros(batch_image_shape)})
            activations = []
            for batch_idx in range(num_batches):
                im_batch_start_idx = int(batch_idx * batch_size)
                im_batch_end_idx = int(np.min([im_batch_start_idx + batch_size, num_images]))
                batch_images = images[im_batch_start_idx:im_batch_end_idx, ...]
                feed_dict = model.get_feed_dict(batch_images, is_test=True)
                feed_dict[model.weight_placeholder] = weights
                outputs = sess.run(activation_operation(), feed_dict)
                activations.append(outputs.copy())
            activations = np.stack(activations, axis=0)
            num_batches, batch_size, num_outputs = activations.shape
            activations = activations.reshape((num_batches*batch_size, num_outputs))
        else:
            feed_dict = model.get_feed_dict(images, is_test=True)
            feed_dict[model.weight_placeholder] = weights
            sess.run(model.init_op, feed_dict)
            activations = sess.run(activation_operation(), feed_dict)
    return activations

In [ ]:
lamb_activation = lambda x : tf.identity(x) # linear
lambda_params = pp.get_params("lambda")
lambda_params.set_data_params("vanhateren")
lambda_params.batch_size = analysis_params.batch_size
lambda_params.data_shape = [analysis_params.patch_edge_size**2] # assumes vector inputs (i.e. not convoultional)
lambda_params.activation_function = lamb_activation
num_neurons_list = [analyzer.model_params.num_neurons for analyzer in analyzers]
linear_activations = []
for num_neurons, lca_weights in zip(num_neurons_list, weights):
    lambda_params.num_neurons = num_neurons
    lambda_model = mp.get_model("lambda")
    lambda_model.setup(lambda_params)
    lambda_activations = compute_lambda_activations(
        data['train'].images[0:num_imgs, ...],
        lambda_model,
        lca_weights,
        batch_size=lambda_params.batch_size
    )
    linear_activations.append(lambda_activations)

In [ ]:
def mask_then_normalize(vector, mask, mask_threshold):
    """
    ensure input is a vector, and then divide it by its l2 norm.
    Parameters:
        mask [np.ndarray] mask to zero out vector values with shape [vector_rows, vector_cols] or [vector_length,]
        vector [np.ndarray] vector with shape [vector_rows, vector_cols] or [vector_length,].
    Outputs:
        vector [np.ndarray] masked vector with shape [vector_length,] and l2-norm = 1
    """
    mask = mask.flatten()
    vector = vector.flatten()
    assert mask.size == vector.size, (
        f'mask size = {mask.size} must equal vector size = {vector.size}')
    mask /= mask.max()
    mask[mask<mask_threshold] = 0
    mask[mask>0] = 1
    vector = np.multiply(mask, vector)
    vector = vector / np.linalg.norm(vector)
    return vector

def angle_between_vectors(vec_a, vec_b):
    """
    Returns the cosine angle between two vectors
    Parameters:
        vec_a [np.ndarray] l2 normalized vector with shape [vector_length, 1]
        vec_b [np.ndarray] l2 normalized vector with shape [vector_length, 1]
    Outputs:
        angle [float] angle between the two vectors, in  degrees
    """
    inner_products = np.dot(vec_a.T, vec_b)
    inner_products = np.clip(inner_products, -1.0, 1.0)
    angle = np.arccos(inner_products) * (180 / np.pi)
    return angle

def one_to_many_angles(vec_a, vec_list_b):
    """
    Returns cosine angle from one vector to a list of vectors
    Parameters:
        vec_a [np.ndarray] l2 normalized vector with shape [vector_length,]
        vec_list_b [list of np.ndarray] list of l2 normalized vectors with shape [num_vectors][vector_length,]
    Outputs:
        angles [list of floats] angle between vec_a and each of the vectors in vec_list_b, in degrees
    """
    angles = []
    for vec_b in vec_list_b:
        angles.append(angle_between_vectors(vec_a, vec_b))
    return angles

def masked_weight_to_image_angles(weight, mask, image_list, mask_threshold):
    """
    """
    num_images = len(image_list)
    vec0 = mask_then_normalize(weight, mask, mask_threshold)
    vec1_list = []
    for image in image_list:
        assert image.size == vec0.size, (
          f'Each image size = {image.size} must equal the weight size = {vec0.size}')
        vec1_list.append(mask_then_normalize(image, mask, mask_threshold))
    angles = one_to_many_angles(vec0, vec1_list)
    return angles

def interesting_image_indices(activations, portion_of_max):
    """
    """
    indices = []
    for activity in activations:
        sub_index_list = []
        for neuron_idx in range(activity.shape[1]):
            threshold = activity[:, neuron_idx].max()*portion_of_max
            sub_index_list.append(np.nonzero((activity[:, neuron_idx] > threshold)))
        indices.append(sub_index_list)
    return indices

In [ ]:
portion_of_max = 0.5
linear_interesting_indices = interesting_image_indices(linear_activations, portion_of_max)
lca_interesting_indices = interesting_image_indices(lca_activations, portion_of_max)

In [ ]:
loop_zip = zip(lca_interesting_indices, linear_interesting_indices, analyzers, model_labels)
for nl_ind, l_ind, analyzer, label in loop_zip:
    num_nl_ind = [neuron_ind[0].size for neuron_ind in nl_ind]
    avg_num_nl_ind = np.mean(num_nl_ind)
    std_num_nl_ind = np.std(num_nl_ind)
    print(f'model {analyzer.model_type}_{analyzer.model_label} had an average of {avg_num_nl_ind:.1f} interesting images')
    num_l_ind = [neuron_ind[0].size for neuron_ind in l_ind]
    avg_num_l_ind = np.mean(num_l_ind)
    std_num_l_ind = np.std(num_l_ind)
    print(f'model linear_{label} had an average of {avg_num_l_ind:.1f} interesting images')
    analyzer.nat_selectivity['num_interesting_img_nl'] = num_nl_ind
    analyzer.nat_selectivity['num_interesting_img_l'] = num_l_ind
    analyzer.nat_selectivity['num_interesting_img_nl_std'] = std_num_nl_ind
    analyzer.nat_selectivity['num_interesting_img_l_std'] = std_num_l_ind
    analyzer.nat_selectivity['num_interesting_img_nl_mean'] = avg_num_nl_ind
    analyzer.nat_selectivity['num_interesting_img_l_mean'] = avg_num_l_ind
    analyzer.nat_selectivity['oc_label'] = label

In [ ]:
num_interesting_means = np.stack([np.array([analyzer.nat_selectivity['num_interesting_img_nl_mean'], analyzer.nat_selectivity['num_interesting_img_l_mean']]) for analyzer in analyzers], axis=0)

df = pd.DataFrame(
    num_interesting_means,
    index=pd.Index(model_labels, name='Overcompleteness'),
    columns=['LCA', 'Linear']
)


fig, ax = plot.subplots(nrows=1, aspect=2, axwidth=4.8, share=0, hratios=(3))
obj = ax.bar(
    df,
    cycle=[color_vals['md_red'], color_vals['md_green']],
    edgecolor='black',
)

num_interesting_stds = np.stack([np.array([analyzer.nat_selectivity['num_interesting_img_nl_std'], analyzer.nat_selectivity['num_interesting_img_l_std']]) for analyzer in analyzers], axis=0)
half_bar_width = np.abs(obj[1].patches[0].xy[0] - obj[0].patches[0].xy[0])/2
lca_bar_locs = [patch.xy[0]+half_bar_width for patch in obj[0].patches]
lin_bar_locs = [patch.xy[0]+half_bar_width for patch in obj[1].patches]
ax.errorbar(lca_bar_locs, num_interesting_means[:,0] , yerr=num_interesting_stds[:,0], color='k', fmt='.')
ax.errorbar(lin_bar_locs, num_interesting_means[:,1] , yerr=num_interesting_stds[:,1], color='k', fmt='.')

ax.legend(obj, frameon=False)
ax.format(
    xlocator=1,
    xminorlocator=0.5,
    ytickminor=False,
    ylim=[0, np.max(num_interesting_means)+np.max(num_interesting_stds)],
    suptitle='Average number of intersting images'
)

In [ ]:
print([analyzer.model_params.num_steps for analyzer in analyzers])

In [ ]:
mask_threshold = 0.5

vect_size = analysis_params.patch_edge_size**2
image_list = [data['train'].images[idx, ...].reshape((vect_size, 1))
    for idx in range(analysis_params.batch_size)]
weight_list = [[weight_matrix[:, idx].reshape((vect_size, 1))
    for idx in range(weight_matrix.shape[1])]
    for weight_matrix in weights]
mask_list = [[envelope.reshape((vect_size, 1))
    for envelope in analyzer.bf_stats['envelopes']]
    for analyzer in analyzers]

for model_index, analyzer in enumerate(analyzers):
    lca_weight_angles = []
    linear_weight_angles = []
    for weight_index in range(analyzers[model_index].model.params.num_neurons):
        model_weight = weight_list[model_index][weight_index]
        model_mask = mask_list[model_index][weight_index]
        lca_images = [data['train'].images[idx, ...].flatten()
            for idx in lca_interesting_indices[model_index][weight_index][0]]
        angles = masked_weight_to_image_angles(model_weight, model_mask, lca_images, mask_threshold)
        lca_weight_angles.append(angles)
        linear_images = [data['train'].images[idx, ...].flatten()
            for idx in linear_interesting_indices[model_index][weight_index][0]]
        angles = masked_weight_to_image_angles(model_weight, model_mask, linear_images, mask_threshold)
        linear_weight_angles.append(angles)
    analyzer.nat_selectivity['lca_angles'] = lca_weight_angles
    analyzer.nat_selectivity['linear_angles'] = linear_weight_angles

In [ ]:
lca_angles = [analyzer.nat_selectivity['lca_angles'] for analyzer in analyzers]
linear_angles = [analyzer.nat_selectivity['linear_angles'] for analyzer in analyzers]
num_plots_per_model = 3
nbins=20
fig, axs = plot.subplots(ncols=len(analyzers), nrows=num_plots_per_model, sharey=False)
max_vals = []
for model_idx in range(len(analyzers)):
    weight_indices = np.random.randint(0, analyzers[model_idx].model_params.num_neurons, num_plots_per_model)
    for row_idx, weight_idx in enumerate(weight_indices):
        indiv_lin_angles = linear_angles[model_idx][weight_idx]
        indiv_lca_angles = lca_angles[model_idx][weight_idx]
        axs[model_idx, row_idx].hist(indiv_lin_angles, bins=nbins, color=color_vals['md_green'], alpha=0.5, label='Linear')
        axs[model_idx, row_idx].hist(indiv_lca_angles, bins=nbins, color=color_vals['md_red'], alpha=0.5, label='LCA')
        max_vals.append(np.max([np.max(indiv_lin_angles), np.max(indiv_lca_angles)]))
        axs[model_idx, row_idx].format(title=f'Neuron {weight_idx}; {model_labels[model_idx]} Overcompleteness')
axs[0,0].legend(loc='ur', ncols=1, frameon=False)
axs.format(suptitle='Exciting image angles per neuron', xlabel='Image-to-weight angle', ylabel='Number of images', xlim=[0, 90])
plot.show()

In [ ]:
for analyzer in analyzers:
    lca_model_means = []
    lca_model_vars = []
    lin_model_means = []
    lin_model_vars = []
    for weight_idx in range(analyzer.model_params.num_neurons):
        indiv_lca_angles = analyzer.nat_selectivity['lca_angles'][weight_idx]
        if len(indiv_lca_angles) > 0:
            lca_model_means.append(np.mean(indiv_lca_angles))
            lca_model_vars.append(np.var(indiv_lca_angles))
        else:
            lca_model_means.append(-1)
            lca_model_vars.append(-1)
        indiv_lin_angles = analyzer.nat_selectivity['linear_angles'][weight_idx]
        if len(indiv_lin_angles) > 0:
            lin_model_means.append(np.mean(indiv_lin_angles))
            lin_model_vars.append(np.var(indiv_lin_angles))
        else:
            lin_model_means.append(-1)
            lin_model_vars.append(-1)
    analyzer.nat_selectivity['lca_means'] = lca_model_means
    analyzer.nat_selectivity['lca_vars'] = lca_model_vars
    analyzer.nat_selectivity['lin_means'] = lin_model_means
    analyzer.nat_selectivity['lin_vars'] = lin_model_vars

In [ ]:
fig, axs = plot.subplots(ncols=len(analyzers), nrows=1, sharey=False)
for ax, analyzer in zip(axs, analyzers):
    lin_data = [mean for mean in analyzer.nat_selectivity['lin_means'] if mean>0]
    non_lin_data = [mean for mean in analyzer.nat_selectivity['lca_means'] if mean>0]
    h1 = ax.hist(lin_data, bins=nbins, color=color_vals['md_green'], alpha=0.5, label='Linear')
    h2 = ax.hist(non_lin_data, bins=nbins, color=color_vals['md_red'], alpha=0.5, label='LCA')
    oc = analyzer.nat_selectivity['oc_label']
    ax.format(title=f'{oc} Overcompleteness')
axs[0,0].legend(loc='ul', frameon=False, ncols=1)
axs[0,0].format(ylabel='Number of images')
axs.format(
    suptitle='Exciting image angles',
    xlabel='Mean image-to-weight angle',
    xlim=[0, 90]
)
plot.show()

In [ ]:
for analyzer in analyzers:
    np.savez(analyzer.analysis_out_dir+'savefiles/natural_image_selectivity.npz', data=analyzer.nat_selectivity)